In [26]:
# Standard includes
import datetime
import re
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from time import time
import timeit
import numpy as np
import pickle

In [27]:
# Define query for the tasks
taskQuery = {
    "query": {
        "bool": {
            "should": [
                {"term":{"reqid": "18663"}}
                #{"term":{"transhome": "AthDerivation-21.2.23.0"}}
            ]
        }
    }
}

In [28]:
# Execute the queries
es = Elasticsearch(['atlas-kibana-dev.mwt2.org'],timeout=120)

# Tasks
taskIndex = "tasks*"
tasks = scan(es, query=taskQuery, index=taskIndex, scroll='5m', timeout="5m", size=1000)        

In [29]:
# Define the trains and the quantities for collection
quantities = [
    'cpuconsumptiontime',
    'nevents',
    'starttime',
    'endtime',
    'timeExe',
    'timeSetup',
    'timeGetJob',
    'timeStageIn',
    'timeStageOut',
    'actualcorecount',
    'wall_time',
    'inputfilebytes',
    'outputfilebytes',
    'Max_PSS_per_core'
]

# used for mc16e validation
trainsAndIDs = [
    (['DAOD_STDM5','DAOD_EGAM4','DAOD_EGAM2','DAOD_EXOT12','DAOD_EXOT9'],[],{}),
    (['DAOD_SUSY12','DAOD_STDM3','DAOD_EXOT15','DAOD_JETM3','DAOD_EXOT19','DAOD_HIGG4D6','DAOD_HIGG6D1','DAOD_HIGG1D1'],[],{}),
    (['DAOD_EXOT22','DAOD_SUSY4','DAOD_JETM11','DAOD_EXOT21','DAOD_STDM7','DAOD_SUSY8','DAOD_SUSY10'],[],{}),
    (['DAOD_HIGG2D1'],[],{}),
    (['DAOD_JETM9','DAOD_STDM4','DAOD_FTAG4'],[],{}),
    (['DAOD_MUON2','DAOD_HIGG4D4','DAOD_JETM7','DAOD_BPHY7','DAOD_EXOT17','DAOD_BPHY5','DAOD_EGAM7','DAOD_HIGG1D2'],[],{}),
    (['DAOD_STDM2','DAOD_SUSY18','DAOD_EXOT3','DAOD_EGAM1','DAOD_EGAM5','DAOD_EXOT2','DAOD_SUSY3','DAOD_EXOT5','DAOD_HIGG6D2'],[],{}),
    (['DAOD_JETM12','DAOD_EGAM3','DAOD_JETM10'],[],{}),
    (['DAOD_TOPQ1'],[],{}),
    (['DAOD_TCAL1','DAOD_EXOT10','DAOD_HIGG2D5'],[],{}),
    (['DAOD_SUSY1'],[],{}),
    (['DAOD_SUSY9'],[],{}),
    (['DAOD_EXOT13','DAOD_SUSY5','DAOD_SUSY7','DAOD_EXOT8','DAOD_EXOT4','DAOD_HIGG4D2'],[],{}),
    (['DAOD_STDM5','DAOD_EGAM4','DAOD_EGAM2','DAOD_EXOT12','DAOD_SUSY9','DAOD_EXOT9'],[],{}),
    (['DAOD_TAUP1','DAOD_HIGG4D5','DAOD_TOPQ5','DAOD_JETM4','DAOD_HIGG4D3','DAOD_SUSY16','DAOD_EXOT7'],[],{}),
    (['DAOD_HIGG8D1','DAOD_JETM6','DAOD_MUON1','DAOD_SUSY6','DAOD_JETM1','DAOD_MUON0','DAOD_TAUP3'],[],{}),
    (['DAOD_EGAM9','DAOD_EXOT20','DAOD_SUSY11','DAOD_EXOT6','DAOD_SUSY2','DAOD_HIGG4D1','DAOD_BPHY1','DAOD_BPHY4'],[],{})
]

# Data structure as follows:
# List of tuples, one per train, with following contents:
# ( [List of carriages in the train],[list of jobs for that train],{map of quantity name->list of values for the jobs})
#trainsAndIDs = [
#    (['DAOD_HIGG2D5','DAOD_TCAL1','DAOD_JETM12'],[],{}), 
#    (['DAOD_EGAM4','DAOD_STDM5','DAOD_EXOT12','DAOD_EXOT10'],[],{}), 
#    (['DAOD_FTAG3'],[],{}), 
#    (['DAOD_EGAM3','DAOD_HIGG1D2','DAOD_EXOT0','DAOD_EXOT17','DAOD_HIGG4D4'],[],{}), 
#    (['DAOD_JETM7','DAOD_EXOT6','DAOD_SUSY9','DAOD_EGAM2','DAOD_SUSY12','DAOD_SUSY2'],[],{}), 
#    (['DAOD_EXOT9','DAOD_EXOT15','DAOD_JETM3','DAOD_HIGG4D1','DAOD_TOPQ2','DAOD_MUON2','DAOD_SUSY16'],[],{}), 
#    (['DAOD_EXOT7','DAOD_SUSY11','DAOD_SUSY9','DAOD_STDM3','DAOD_TOPQ5','DAOD_EXOT19'],[],{}), 
#    (['DAOD_HIGG1D1','DAOD_EGAM9','DAOD_BPHY5','DAOD_BPHY4','DAOD_JETM4','DAOD_SUSY18'],[],{}), 
#    (['DAOD_EGAM7','DAOD_HIGG6D1','DAOD_STDM2','DAOD_SUSY3','DAOD_EGAM1'],[],{}), 
#    (['DAOD_EXOT22','DAOD_SUSY1','DAOD_JETM1','DAOD_EXOT3','DAOD_SUSY4','DAOD_BPHY1'],[],{}), 
#    (['DAOD_JETM11','DAOD_HIGG8D1','DAOD_EXOT5','DAOD_SUSY5'],[],{}), 
#    (['DAOD_SUSY10','DAOD_STDM7','DAOD_JETM6','DAOD_JETM9','DAOD_TAUP1','DAOD_SUSY7','DAOD_HIGG4D5'],[],{}), 
#    (['DAOD_EXOT4','DAOD_HIGG4D6','DAOD_SUSY8','DAOD_HIGG4D2','DAOD_MUON1'],[],{}), 
#    (['DAOD_EXOT8','DAOD_HIGG4D3','DAOD_EXOT2'],[],{}), 
#    (['DAOD_HIGG2D1','DAOD_SUSY6','DAOD_EXOT13'],[],{}), 
#    (['DAOD_FTAG1','DAOD_FTAG2','DAOD_FTAG4'],[],{}), 
#    (['DAOD_TOPQ1','DAOD_EGAM5','DAOD_STDM4'],[],{}), 
#    (['DAOD_HIGG6D2','DAOD_EXOT21','DAOD_MUON0','DAOD_TAUP3','DAOD_TOPQ4'],[],{})
#]

In [30]:
# Get the IDs matching each train

start_time = timeit.default_timer()
matchingTasks = 0
for res in tasks:
    if 'output_formats' in res['_source'].keys():
        for item in trainsAndIDs:
            train = item[0]
            if set(res['_source']['output_formats']) == set(train):
                item[1].append(res['_id'])
                matchingTasks += 1
print("Total matching tasks = ",matchingTasks)
print("Time to extract information = ",timeit.default_timer() - start_time)

Total matching tasks =  51
Time to extract information =  0.04285682737827301


In [31]:
# Save the per-train information
pickle.dump(trainsAndIDs,open("data_21_2_23_0.p","wb"))

In [32]:
trainsAndIDs

[(['DAOD_STDM5', 'DAOD_EGAM4', 'DAOD_EGAM2', 'DAOD_EXOT12', 'DAOD_EXOT9'],
  ['15171292', '15171272', '15171253'],
  {}),
 (['DAOD_SUSY12',
   'DAOD_STDM3',
   'DAOD_EXOT15',
   'DAOD_JETM3',
   'DAOD_EXOT19',
   'DAOD_HIGG4D6',
   'DAOD_HIGG6D1',
   'DAOD_HIGG1D1'],
  ['15084442', '15084494', '15084467'],
  {}),
 (['DAOD_EXOT22',
   'DAOD_SUSY4',
   'DAOD_JETM11',
   'DAOD_EXOT21',
   'DAOD_STDM7',
   'DAOD_SUSY8',
   'DAOD_SUSY10'],
  ['15084704', '15084731', '15084679'],
  {}),
 (['DAOD_HIGG2D1'], ['15085014', '15085031', '15085003'], {}),
 (['DAOD_JETM9', 'DAOD_STDM4', 'DAOD_FTAG4'],
  ['15084913', '15084932', '15084947'],
  {}),
 (['DAOD_MUON2',
   'DAOD_HIGG4D4',
   'DAOD_JETM7',
   'DAOD_BPHY7',
   'DAOD_EXOT17',
   'DAOD_BPHY5',
   'DAOD_EGAM7',
   'DAOD_HIGG1D2'],
  ['15084315', '15084254', '15084280'],
  {}),
 (['DAOD_STDM2',
   'DAOD_SUSY18',
   'DAOD_EXOT3',
   'DAOD_EGAM1',
   'DAOD_EGAM5',
   'DAOD_EXOT2',
   'DAOD_SUSY3',
   'DAOD_EXOT5',
   'DAOD_HIGG6D2'],
  ['15084649